In [ ]:
from IPython.display import display
import pandas as pd

from ev_charger_reliability.pipelines.preprocess_data.nodes import preprocess_charging_sessions

%reload_kedro
session.run()

In [ ]:
x = catalog.load("charging_sessions_preprocessed")